# Subsetting the data

## Acerca de los datos
En este cuaderno trabajaremos con datos de terremotos del 18 de septiembre de 2018 al 13 de octubre de 2018 (obtenidos del Servicio Geológico de Estados Unidos (USGS) mediante la [API del USGS](https://earthquake.usgs.gov/fdsnws/event/1/))

## Configuración
Estaremos trabajando con el archivo `data/earthquakes.csv` nuevamente, por lo que necesitamos manejar nuestras importaciones y leerlo.

In [ ]:
import pandas as pd

df = pd.read_csv('data/earthquakes.csv')

## Seleccionar columnas
Coge una columna entera usando la notación de atributos:

In [ ]:
df.mag

Coge una columna entera utilizando la sintaxis de diccionario:

In [ ]:
df['mag']

Selección de varias columnas:

In [ ]:
df[['mag', 'title']]

Selección de columnas mediante comprensión de listas y operaciones con cadenas:

In [ ]:
df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

Desglosando este ejemplo:
1. la comprensión de la lista

In [ ]:
[col for col in df.columns if col.startswith('mag')]

2. confección de la lista

In [ ]:
['title', 'time'] \
+ [col for col in df.columns if col.startswith('mag')]

3. utilizar esta lista como lista de columnas

In [ ]:
df[
    ['title', 'time']
    + [col for col in df.columns if col.startswith('mag')]
]

## Slicing
### Seleccionar filas
Utilizando números de fila (incluido el primer índice, excluido el último):

In [ ]:
df[100:103]

### Selección de filas y columnas con encadenamiento

In [ ]:
df[['title', 'time']][100:103]

El orden no importa aquí:

In [ ]:
df[100:103][['title', 'time']].equals(
    df[['title', 'time']][100:103]
)

Ya sabemos cómo seleccionar filas y columnas, pero ¿podemos actualizar valores? Bueno, si intentamos utilizar lo que hemos aprendido hasta ahora, veremos la siguiente advertencia:

In [ ]:
df[110:113]['title'] = df[110:113]['title'].str.lower()

Fíjate que aquí funcionó, pero `pandas` dice que estábamos estableciendo un valor en una copia de un slice y que deberíamos usar `loc` en su lugar (tema de la siguiente sección):

In [ ]:
df[110:113]['title']

## Indexación

Ahora si hacemos esto con `loc` como sugiere la advertencia, todo va como la seda. Nótese que tenemos que bajar el índice final en uno ya que `loc` incluye los puntos finales:

In [ ]:
df.loc[110:112, 'title'] = df.loc[110:112, 'title'].str.lower()
df.loc[110:112, 'title']

### Indexación con `loc`
Selección del formato `loc[indexador_filas, indexador_columnas]` donde `:` puede utilizarse para seleccionar todo:

In [ ]:
df.loc[:,'title']

Podemos utilizar `loc` para seleccionar filas y columnas específicas sin encadenar. Si usamos números de fila con `loc`, ahora son **inclusivos** del índice final:

In [ ]:
df.loc[10:15, ['title', 'mag']]

#### Indexación con `iloc`
Exclusivo del punto final al igual que Python slicing:

In [ ]:
df.iloc[10:15, [19, 8]]

Podemos utilizar la sintaxis de corte con `iloc` tanto para filas como para columnas:

In [ ]:
df.iloc[10:15, 6:10]

Cuando usamos `loc`, podemos hacer cortes en los nombres de las columnas. Esto incluirá el punto final porque no se puede esperar saber cuál será el siguiente nombre de columna. Como tal, tenemos varias maneras de lograr el mismo objetivo final:

In [ ]:
df.iloc[10:15, 6:10].equals(
    df.loc[10:14, 'gap':'magType']
)

### Buscando valores escalares
Hemos utilizado `loc` y `iloc` para obtener subconjuntos del marco de datos. Sin embargo, si sólo estamos interesados en el valor específico en una determinada `[fila, columna]`, entonces podemos utilizar `iat` y `at`. Usamos `at` con etiquetas:

In [ ]:
df.at[10, 'mag']

...y `iat` con índices enteros:

In [ ]:
df.iat[10, 8]

## Filtrado
Podemos filtrar nuestros marcos de datos utilizando una **máscara booleana**, que se puede hacer de la siguiente manera:

In [ ]:
df.mag > 2

Para utilizar una máscara de selección, basta con colocarla dentro de los corchetes:

In [ ]:
df[df.mag >= 7.0]

Podemos utilizar máscaras con `loc`:

In [ ]:
df.loc[
    df.mag >= 7.0,
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Se pueden crear máscaras utilizando múltiples criterios cuando se combinan con los operadores bit a bit `&` para AND y `|` para OR. También debemos rodear cada criterio con paréntesis. No podemos usar `and`/`or` aquí porque necesitamos evaluar fila por fila:

In [ ]:
df.loc[
    (df.tsunami == 1) & (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Un ejemplo con una condición OR, que es menos restrictiva:

In [ ]:
df.loc[
    (df.tsunami == 1) | (df.alert == 'red'),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Se pueden crear máscaras a partir de cualquier criterio que dé como resultado un booleano. Por ejemplo, podemos seleccionar todos los terremotos con la cadena `Alaska` en la columna `place` con un valor no nulo para la columna `alert`. Para obtener valores no nulos, podemos utilizar el método `isnull()` con el operador de negación bitwise (`~`) o el método `notnull()`:

In [ ]:
df.loc[
    (df.place.str.contains('Alaska')) & (df.alert.notnull()),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Aquí podemos utilizar incluso expresiones regulares:

In [ ]:
df.loc[
    (df.place.str.contains(r'CA|California$')) & (df.mag > 3.8),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Podemos utilizar el método `between()` para convertir 2 comprobaciones individuales (es menor o igual que algún valor máximo y es mayor o igual que algún valor mínimo) en una sola. Tenga en cuenta que esto incluye el punto final por defecto:

In [ ]:
df.loc[
    df.mag.between(6.5, 7.5),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Podemos utilizar el método `isin()` para comprobar la pertenencia a una lista de valores:

In [ ]:
df.loc[
    df.magType.isin(['mw', 'mwb']),
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Podemos obtener el índice de los valores mínimo y máximo de una columna determinada y utilizarlos para seleccionar toda la fila en la que aparecen:

In [ ]:
[df.mag.idxmin(), df.mag.idxmax()]

In [ ]:
df.loc[
    [df.mag.idxmin(), df.mag.idxmax()],
    ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

Tenga en cuenta que hay un método `filter()`, pero no filtra los datos en el mismo sentido que hemos discutido en esta sección. Aquí hay algunas cosas que puedes hacer con este método.

- agarrar columnas de un dataframe pasando una lista a `items`:

In [ ]:
df.filter(items=['mag', 'magType']).head()

- coge todas las columnas que contengan una cadena con el parámetro `like`:

In [ ]:
df.filter(like='mag').head()

- utilizar expresiones regulares; aquí, seleccionamos cualquier columna que empiece por `t`:

In [ ]:
df.filter(regex=r'^t').head()

- utilizar `filter()` a lo largo de las filas, pasando en `axis=0`. Aquí utilizaremos la columna `place` como índice (veremos `set_index()` en el capítulo 3):

In [ ]:
df.set_index('place').filter(like='Japan', axis=0).filter(items=['mag', 'magType', 'title']).head()

Esto también funciona con objetos `Series` y se ejecutará en el índice:

In [ ]:
df.set_index('place').title.filter(like='Japan').head()

<hr>
<div>
    <a href="./4-inspeccionando_dataframes.ipynb">
        <button style="float: left;">&#8592; Notebook Anterior</button>
    </a>
    <a href="./6-anadiendo_y_eliminando_data.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>